# Project Summary
NooraGPT is an AI-powered assistant designed to handle internal queries related to the Home Salon by Nooora business operations. It leverages a Neo4j graph database and a local open-source LLM (Ollama + Qwen 2.5) to generate natural responses for business-related questions.

In [1]:
import asyncio
import uuid
from langchain_core.runnables import RunnableSequence
from py2neo import Graph
from langchain_ollama import ChatOllama

from langchain.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate
)

In [2]:
graph = Graph("bolt://localhost:7687", auth=("neo4j", "12345678"))

llm = ChatOllama(model="qwen2.5", temperature=0.1, streaming = True)

In [3]:
labels = """
- Employee /
- Department /
- Skill /
- Customer /
- Vehicle /
- Task /
"""

relationships = """
- (Employee)-[:WORKS_IN]->(Department) /
- (Employee)-[:HAS_SKILL]->(Skill) /
- (Task)-[:HAS_CUSTOMER]->(Customer) /
- (Task)-[:HAS_DRIVER]->(Employee) /
- (Task)-[:USES_VEHICLE]->(Vehicle) /
- (Task)-[:SELECTED_TECHNICIAN]->(Employee) /
"""

departments = """
- Administration /
- Unknown Department /
- Technician Team /
- Drivers /
- Customer Support Team /
- Human Resource Department /
"""

employee_properties = """
- contact: str /
- email: str /
- gender: str /
- id: int /
- job_title: str /
- name: str /
- private_address: str /
- skill_tags: list /
- working_shift: str /
"""

customer_properties = """
- address: str /
- contact: str /
- email: str /
- gender: str /
- id: int /
- map_url: str /
- name: str /
"""

vehicle_properties = """
- id: int /
- license_plate: str /
- name: str /
"""


In [4]:
system_prompt = f"""
You are a helpful Cypher query generator. Your task is to generate only the Cypher query in response to user questions about a Neo4j graph.
Do not include explanations or comments in your response.

The schema is as follows:
--------------------------
The Neo4j graph has the following node labels: {labels}
The following relationships: {relationships}
The 'Department' nodes are as follows: {departments}
The 'Employee' properties are: {employee_properties}
The 'Vehicle' properties are: {vehicle_properties}
----------------------------

Here are some examples:
Example 1: For the question "Give email address of customer whose id is 96982"
MATCH (c:Customer {{{{id: 96982}}}})
RETURN c.email

Example 2: For the question "Who is the HR?"
MATCH (e:Employee)-[:WORKS_IN]->(d:Department {{{{name: 'Human Resource Department'}}}})
RETURN e.name

Example 3: For the question "Give contact number of employee whose id is 462"
MATCH (e:Employee {{{{id: 462}}}})
RETURN e.contact

"""

In [5]:
# 2. Prompt for generating Cypher
generate_cypher_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_prompt),
    HumanMessagePromptTemplate.from_template("{query}"),
])

In [6]:
# 3. Function to run Cypher on Neo4j
def run_cypher(cypher: str) -> dict:
    result = graph.run(cypher).data()
    return {"results": result}

# 4. Prompt for final answer generation
def generate_natural_language_prompt(data: dict) -> str:
    return f"""
You are NooraGPT, a helpful assistant for 'Home Salon by Nooora', which provides home salon services in Dubai.  
You are given a user question and the answer fetched from a graph database.  
Your task is to write a short, natural response for the user.

If the user’s question is general or unrelated to Noora's internal operations, and the Database Result is empty, 
politely inform them that you can only answer questions about Noora’s internal information.

The database contains the following labels: {labels}

Question: ```{data['user']}```
Database Result: ```{data['results']}```
"""



In [7]:
# 5. Build the pipeline
pipeline = (
    generate_cypher_prompt
    | llm
    | (lambda msg: {"user": user_question, "cypher": msg.content})
    | (lambda d: {**d, **run_cypher(d["cypher"])})
    | (lambda d: generate_natural_language_prompt(d))
    | llm
)

# 6. Run it
user_question = "who is HR"
final_response = pipeline.invoke({"query": user_question})
print("\n💬 Final Answer:\n", final_response.content)



💬 Final Answer:
 Hello! The HR representative at Noora is Pranali Ramkrishna Kadam. If you have any questions or need assistance related to our services, feel free to reach out! 😊


# User Streaming

In [8]:
pipeline = (
    generate_cypher_prompt
    | llm
    | (lambda msg: {
        "user": msg.additional_kwargs.get("query"),  # extract saved query
        "cypher": msg.content
      })
    | (lambda d: {**d, **run_cypher(d["cypher"])})
    | (lambda d: generate_natural_language_prompt(d))
    | llm
)

In [9]:
async def chat_loop():
    session_id = str(uuid.uuid4())
    print("NooraGPT is ready to assist you with internal operations.\nType 'q' or 'quit' to exit.\n")

    while True:
        user_input = input("🧑 You: ")
        if user_input.lower() in ["q", "quit"]:
            print("👋 NooraGPT: Goodbye!")
            break

        print("🤖 NooraGPT: ", end="", flush=True)

        async for chunk in pipeline.astream(
            {"query": user_input},  # query used for prompt
            config={"configurable": {"session_id": session_id}}
        ):
            print(chunk.content, end="", flush=True)
        print()


In [10]:
await chat_loop()

NooraGPT is ready to assist you with internal operations.
Type 'q' or 'quit' to exit.



🧑 You:  who is hr


🤖 NooraGPT: Hello! The information I found in our records shows that the employee name is Pranali Ramkrishna Kadam. If you have any specific questions about her or need more details, feel free to ask!


🧑 You:  q


👋 NooraGPT: Goodbye!
